In [3]:
import Model2Fuzzy as mf
import pandas as pd
import spacy
# Texto que deseas poner en el DataFrame
texto1 = "se produce una inundación en ave garza sada y av del estado, cerca de la colonia villa florida en mty, nl, bomberosmty"
texto2 = "precaucion por lluvias fuertes en la colonia del valle y la calle rio misisipi"
# Crear un DataFrame con una columna de texto
df = pd.DataFrame({'Texto': [texto1, texto2]})


lables = ["COL", "CALLE", "MUN", "REGION"]
model = spacy.load("fuzzymodelV1_0_2")
outputname = "text.csv"
newdf = mf.CreateLabels(model, df, "Texto", lables, outputname)
newdf = pd.read_csv(outputname)

colIterableNames = ["municipio", "colonia", "calle"]
realData = pd.read_csv("DataMTY.csv")
outputname = "textGOD1.csv"
testdf = mf.Fuzzy2Result(newdf, realData, colIterableNames, outputname)
testdf = pd.read_csv(outputname)



c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Creating labels
Labels created
Analyzing coincidences
Fila 0: Reemplazando 'monterrey' por 'Monterrey' con 100% de similitud
Fila 0: Reemplazando 'villaflorida' por 'VILLA FLORIDA' con 100% de similitud
Fila 1: Reemplazando 'delvalle' por 'DEL VALLE' con 100% de similitud
Fila 0: Reemplazando 'eugeniogarzasada' por 'Avenida Eugenio Garza Sada' con 100% de similitud
Fila 0: Reemplazando 'avdelestado' por 'Avenida del Estado' con 90% de similitud
Fila 1: No se encontró coincidencia suficiente para 'riomisisipi' (Similitud: 64%)
Process finished. Fuzzy dataframe created.


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch


# Cargar el tokenizador y el modelo
tokenizer = BertTokenizer.from_pretrained('./class_my_model')
model = BertForSequenceClassification.from_pretrained('./class_my_model')

# Cargar el LabelEncoder (Asegúrate de que es el mismo usado durante el entrenamiento)
df_original = pd.read_csv("tweets_better_classified.csv")  # Dataset original para recuperar las etiquetas
label_encoder = LabelEncoder()
label_encoder.fit(df_original['Tipo'])  # Ajusta 'Tipo' a la columna de etiquetas original

# Cargar el dataset filtrado
df_filtered = pd.read_csv("textGOD1.csv")

# Define una función para predecir nuevos textos
def predict_new_text(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Asegúrate de mover el modelo y los inputs a la GPU si está disponible
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Hacer la predicción
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Obtener la clase predicha
    predicted_class = torch.argmax(logits, dim=1).item()

    # Decodificar la etiqueta predicha
    predicted_label = label_encoder.inverse_transform([predicted_class])

    return predicted_label[0]

# Predecir etiquetas para la columna 'texto' y almacenarlas en una nueva columna 'Tipo'
df_filtered['Tipo'] = df_filtered['Texto'].apply(predict_new_text)

# Guardar el nuevo DataFrame en un archivo CSV
df_filtered.to_csv("textGOD2.csv", index=False)

print("Archivo 'tweets_classified.csv' guardado con las etiquetas predichas.")

Archivo 'tweets_classified.csv' guardado con las etiquetas predichas.


c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd

# Cambia el modelo a uno adecuado para español
MODEL = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Lee el archivo CSV
file_path = 'textGOD2.csv'
df = pd.read_csv(file_path)

# Asignación de valores a las etiquetas de sentimiento
sentiment_labels = [-2, -1, 0, 1, 2]

# Función para calcular la puntuación de sentimiento
def calculate_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    scores = outputs.logits.detach().numpy()
    scores = softmax(scores[0])
    sentiment_score = sum([a*b for a, b in zip(sentiment_labels, scores)])
    return sentiment_score

# Aplicar la función a cada texto en la columna 'Texto'
df['Puntaje'] = df['Texto'].apply(calculate_sentiment)



# Guardar el dataset modificado en un nuevo archivo CSV
output_path = 'textGOD3.csv' 
df.to_csv(output_path, index=False)

c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from geopy.geocoders import Nominatim
import time
import ast

def safe_convert_to_list(value):
    """
    Convierte un valor de texto que parece una lista en una lista de Python.
    Si ya es una lista, la devuelve tal cual.
    Si no es posible convertirlo, devuelve una lista vacía.
    """
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            # Intentar convertir la cadena a una lista si tiene el formato adecuado
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            # Si no es posible, devolver una lista vacía
            return [value]  # Devolver la cadena como un elemento en la lista
    return []

# Inicializar el geocodificador
geolocator = Nominatim(user_agent="mi_aplicacion_mapa")

# Cargar el archivo SHP de municipios
shapefile_municipios = r"2023_1_19_A\2023_1_19_A.shp"
gdf_municipios = gpd.read_file(shapefile_municipios)

# Cargar el archivo SHP de colonias
shapefile_colonias = r"Colonias\Colonias.shp"
gdf_colonias = gpd.read_file(shapefile_colonias)
gdf_colonias = gdf_colonias[gdf_colonias["ST_NAME"] == "NUEVO LEON"]

# Cargar el DataFrame principal con los datos desde B_testDf.csv
df_calles = pd.read_csv("textGOD3.csv")

# Excluir los municipios "Monte Morelos" y "Saltillo"
df_calles = df_calles[~df_calles['municipio'].isin(['monte morelos', 'saltillo'])]

# Filtrar los municipios y calcular el puntaje promedio por municipio
cve_mun_values = {
    '006': 'apodaca',
    '019': 'san pedro garza garcía',
    '021': 'escobedo',
    '026': 'guadalupe',
    '039': 'monterrey',
    '046': 'san nicolás',
    '048': 'santa catarina',
    '049': 'santiago'
}

promedio_puntajes_municipios = {}
for cve, municipio in cve_mun_values.items():
    filas_municipio = df_calles[df_calles['municipio'].str.lower() == municipio.lower()]
    if not filas_municipio.empty:
        promedio_puntajes_municipios[cve] = np.mean(filas_municipio['Puntaje'])
    else:
        promedio_puntajes_municipios[cve] = None

# Asignar los puntajes promedios a los códigos CVE_MUN correspondientes
gdf_municipios['Puntaje'] = gdf_municipios['CVE_MUN'].map(promedio_puntajes_municipios)

# Configurar el colormap para municipios
norm_municipios = Normalize(vmin=-2, vmax=2)
cmap_municipios = cm.get_cmap('RdYlGn')

# Crear el mapa centrado en Monterrey
mapa = folium.Map(location=[25.6866, -100.3161], zoom_start=13)

# Crear capas para municipios, colonias y calles
municipios_layer = folium.FeatureGroup(name='Municipios')
colonias_layer = folium.FeatureGroup(name='Colonias')
calles_layer = folium.FeatureGroup(name='Calles')  # Nueva capa para las calles

# Añadir los polígonos de municipios al mapa
for _, row in gdf_municipios.iterrows():
    puntaje = row['Puntaje']
    if puntaje is not None:
        color = cmap_municipios(norm_municipios(puntaje))[:3]
        color = [int(c * 255) for c in color]
        folium.GeoJson(
            row['geometry'],
            style_function=lambda x, color=color: {
                'fillColor': f'rgba({color[0]}, {color[1]}, {color[2]}, 0.6)',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.6,
            }
        ).add_to(municipios_layer)

# Añadir los polígonos de colonias al mapa
df_calles['colonia'] = df_calles['colonia'].astype(str).str.upper()
gdf_colonias = gdf_colonias.merge(df_calles, left_on='SETT_NAME', right_on='colonia', how='left')

norm_colonias = Normalize(vmin=df_calles['Puntaje'].min(), vmax=df_calles['Puntaje'].max())
cmap_colonias = cm.get_cmap('RdYlGn')

for _, row in gdf_colonias.iterrows():
    puntaje = row['Puntaje']
    if puntaje is not None:
        color = cmap_colonias(norm_colonias(puntaje))[:3]
        color = [int(c * 255) for c in color]
        folium.GeoJson(
            row['geometry'],
            style_function=lambda x, color=color: {
                'fillColor': f'rgba({color[0]}, {color[1]}, {color[2]}, 0.6)',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.4,
            }
        ).add_to(colonias_layer)

# Lista de municipios a probar en caso de que no se encuentre la calle en Monterrey
municipios_lista = ['Monterrey', 'Apodaca', 'San Pedro Garza García', 'Escobedo', 'Guadalupe', 'San Nicolás', 'Santa Catarina', 'Santiago']

# Iterar sobre las calles y listas de calles
for idx, row in df_calles.iterrows():
    calles_lista = row['calle']
    puntaje = row['Puntaje']  # Obtener el puntaje de la calle
    tipo = row['Tipo']  # Obtener la clasificación de la calle
    
    # Definir el ícono y el color según el tipo de calle
    if tipo.lower() == 'lluvia':
        icono = "cloud"
        color_icono = "blue"
    elif tipo.lower() == 'inundación':
        icono = "tint"
        color_icono = "red"
    elif tipo.lower() == 'eléctrico':
        icono = "flash"  # Cambiado a 'flash' ya que 'plug' no está disponible
        color_icono = "orange"
    else:
        icono = "question-circle"
        color_icono = "gray"  # Color para los tipos no relacionados
    
    # Asegurarse de que el valor de 'calle' es una cadena
    if isinstance(calles_lista, str):
        # Dividir por comas en caso de que haya más de una calle en la celda
        calles_lista = [calle.strip() for calle in calles_lista.split(',')]
    
    # Iterar sobre la lista de calles y geolocalizarlas
    if isinstance(calles_lista, list) and calles_lista:
        for calle in calles_lista:
            if calle:  # Asegurarse de que la calle no esté vacía
                found_location = False
                for municipio in municipios_lista:
                    try:
                        # Intentar geolocalizar la calle
                        location = geolocator.geocode(calle + ", " + municipio + ", Mexico")
                        time.sleep(1)  # Evitar sobrecarga del servicio
                        if location:
                            # Crear popup con información de la calle
                            popup_text = f"<b>Calle:</b> {calle}<br><b>Puntaje:</b> {puntaje}<br><b>Clasificación:</b> {tipo}"
                            # Añadir el marcador con el ícono correspondiente
                            folium.Marker(
                                location=[location.latitude, location.longitude],
                                popup=popup_text,
                                icon=folium.Icon(icon=icono, color=color_icono),  # Añadir ícono y color según la clasificación
                            ).add_to(calles_layer)  # Añadir marcador a la capa de calles
                            found_location = True
                            break
                    except Exception as e:
                        print(f"Error al intentar obtener las coordenadas para {calle} en {municipio}: {e}")
                
                if not found_location:
                    print(f"No se encontraron coordenadas para {calle} en ningún municipio.")

# Añadir las capas al mapa
mapa.add_child(municipios_layer)
mapa.add_child(colonias_layer)
mapa.add_child(calles_layer)  # Añadir la capa de calles

# Añadir control de capas
folium.LayerControl().add_to(mapa)

# Guardar el mapa como archivo HTML
mapa.save("mapa4.html")

print("Mapa guardado como 'mapa4.html'.")

C:\Users\lol_a\AppData\Local\Temp\ipykernel_22200\3035917485.py:71: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap_municipios = cm.get_cmap('RdYlGn')
C:\Users\lol_a\AppData\Local\Temp\ipykernel_22200\3035917485.py:102: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap_colonias = cm.get_cmap('RdYlGn')


Mapa guardado como 'mapa4.html'.
